In [51]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
import sys


In [52]:
filename = 'courseRecSys.xlsx'
splitProp = 0.95

In [53]:
def read_data(filename):
    df = pd.read_excel(filename,
    header=0,
    index_col=False,
    keep_default_na=True
    )
    return df

In [54]:
def preprocess(df):
    df=df.drop(['Start time','Completion time','Email','Name','Name(not mandatory)\n','ID'], axis=1)
    df["avg"] = df.mean(axis = 0, skipna=True, numeric_only=True)
    df=df.apply(lambda x: x.fillna(x.mean()),axis=0)
    df = df.drop(["avg"],axis =1)
    return df

In [55]:
df=read_data(filename)
df = preprocess(df)
df

,Data Structures and Algorithms,Computer Architecture\n,Discrete Mathematics\n,Economics,Programming-2,Machine Learning\n,Mathematics For Machine Learning\n,Visual Recognition\n,Natural Language Processing\n,Reinforcement Learning,...,Digital CMOS VLSI Design,System design with FPGA,ASIC design,VLSI Architecture Design\n,High level synthesis and optimization of Digital Circuits,Digital Sociology,Privacy in the Digital Age,Technology Ethics and AI,Techno-economics of networks,The Web and the Mind
0,0.30,0.85,0.85,0.90,0.70,0.70,0.80,0.620098,0.572638,0.603495,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,0.600000,0.423286,0.396622,0.510897
1,0.70,0.80,0.20,0.80,0.80,0.50,0.10,0.200000,0.572638,0.603495,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,0.500000,0.423286,0.396622,0.510897
2,0.70,0.00,1.00,1.00,1.00,1.00,1.00,0.800000,0.900000,1.000000,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,0.400000,0.423286,0.396622,0.510897
3,0.65,0.70,0.20,0.75,0.75,0.75,0.75,0.750000,0.572638,0.603495,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.800000,0.452708,0.423286,0.800000,0.510897
4,0.70,0.60,0.20,0.00,0.80,0.50,0.60,0.400000,0.400000,0.200000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,0.00,1.00,0.50,0.70,0.60,0.50,0.30,0.700000,0.572638,0.603495,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,0.000000,0.423286,0.396622,0.510897
158,1.00,0.00,0.00,0.70,0.70,0.00,0.00,0.000000,0.572638,0.603495,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,0.452708,0.423286,0.396622,0.200000
159,0.60,0.30,0.40,0.90,0.80,0.70,0.40,0.500000,0.400000,0.200000,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,1.000000,0.423286,0.396622,1.000000
160,0.80,0.60,0.20,0.40,0.70,0.70,0.80,0.620098,0.572638,0.700000,...,0.251538,0.246316,0.214805,0.234868,0.168056,0.318243,0.452708,0.423286,0.396622,0.510897


change 0.5 to average of ratings of that user

In [56]:
all_users = df

In [57]:
train,test = train_test_split(df,train_size=splitProp, random_state=42,shuffle = True)

In [58]:
courses = df.columns
# courses

In [59]:
core_courses = ['Data Structures and Algorithms','Computer Architecture\n','Discrete Mathematics\n','Economics','Programming-2']

In [60]:
def CosineSimilarity(vec1,vec2):
    return np.dot(vec1,vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec1))


new user with just core courses ratings. Now we need elective courses

In [61]:
new_user = [0.2,0.8,0.2,0.6,0.9]
new_user2 =[ 0.699459,	0.545405	,0.325405	,0.615135	,0.737838]
sooraj = [0.4, 0.8,0.85,0.9,0.7]
chinmay = [0.9,0.2,0.4,0.6 ,0.8]
kritin = [0,0,0,0,0]

In [62]:
core_courses

['Data Structures and Algorithms',
 'Computer Architecture\n',
 'Discrete Mathematics\n',
 'Economics',
 'Programming-2']

In [63]:
def findSimilarUsers(all_users,new_user):
    similarity_user_cosines = []

    for user in all_users:
        mean = np.mean(user)
        # theta = CosineSimilarity(user[0:5],new_user)
        theta = CosineSimilarity(user[0:5]-mean,new_user)

        similarity_user_cosines.append(theta)
    return similarity_user_cosines
    

In [64]:
check = findSimilarUsers(train.to_numpy(),new_user)
len(check)

153

In [65]:
all_users = all_users.to_numpy()

In [66]:
min = findSimilarUsers(all_users,new_user)

# ranking of electives

In [67]:
all_users.shape,len(min)
train.shape



(153, 25)

In [68]:
d={tuple(all_users[i]):min[i] for i in range(len(all_users))}


In [69]:
def getClosestUsers(min,all_users):
    users_ranked = sorted(all_users,key=lambda x:d[tuple(x)])
    return users_ranked

In [70]:
def getUserRating(min,all_users,new_user):
    users_ranked = getClosestUsers(min,all_users)
    user_rating = np.zeros(25)
    for i in range(5):
        user_rating = np.add(users_ranked[i],user_rating)
    user_rating = user_rating/5
    mean = np.ones(25)*np.mean(new_user)#new method

    user_rating = np.add(user_rating,mean) #new method
    return user_rating

In [71]:
user_rating = getUserRating(min,all_users,new_user)

In [72]:
def get_electives_ranked(courses,elective_ratings):
    d={courses[5:][i]: elective_ratings[i] for i in range(20)}

    elective_ranked = sorted(courses[5:], key=lambda x : -d[x])
    return elective_ranked  

In [73]:
user_elective_ranked=get_electives_ranked(courses,user_rating)
user_elective_ranked

['Software Production Engineering\n',
 'Computer Graphics\n',
 'Digital CMOS VLSI Design ',
 'Data Visualization\n',
 'Cloud Computing\n',
 'Programming Languages\n',
 'Machine Learning\n',
 'Visual Recognition\n',
 'VLSI Architecture Design\n',
 'High level synthesis and optimization of Digital Circuits ',
 'ASIC design ',
 'System design with FPGA ',
 'Reinforcement Learning',
 'Mathematics For Machine Learning\n',
 'Digital Sociology ',
 'Privacy in the Digital Age',
 'Techno-economics of networks ',
 'Natural Language Processing\n',
 'Technology Ethics and AI ',
 'The Web and the Mind ']